In [ ]:
## In a previous notebook, I have extracted the audio files to wav files, and dumped them into 
# train_extracted_audio and test_extracted_audio

## Data set was loaded from huggingface
# Load the dataset
#dataset = load_dataset("Hani89/medical_asr_recording_dataset")
#Hani89/medical_asr_recording_dataset

In [6]:
import pandas as pd

train_df = pd.read_csv("train_extracted_audio/metadata.csv")
test_df = pd.read_csv("test_extracted_audio/metadata.csv")
print(f"Loading {len(train_df)} training rows and {len(test_df)} test rows")

Loading 5328 training rows and 1333 test rows


In [7]:
## Will load the two data sets independently, but want to track the training and test splits
train_df['split'] = 'train'
test_df['split'] = 'test'
train_df['filePath'] = "train_extracted_audio/" + train_df['filename']
test_df['filePath'] = "test_extracted_audio/" + test_df['filename']


In [1]:
import os
os.environ['PIXELTABLE_PGDATA'] = '/scr/dagutman/devel/messageAnalytics/.pxtData'

import pixeltable as pxt

In [ ]:
## Create / initialize pxt table

pxt.drop_dir('hani89_asr_data', if_not_exists="ignore", force=True) 
pxt.create_dir('hani89_asr_data',if_exists='ignore')

## Create a table to store audio files
pxt.create_table('hani89_asr_data.transcribe_compare',
                  {'id': pxt.String,
                   'audio': pxt.Audio,
                   'filename': pxt.String,
                   'transcription': pxt.String,
                   'split': pxt.String,
                   'filePath': pxt.String}
                  )

In [8]:
## Load the training data frame
t = pxt.get_table('hani89_asr_data.transcribe_compare')
t.insert(train_df.to_dict(orient='records')[500:2500])

Inserting rows into `transcribe_compare`: 2000 rows [00:02, 846.55 rows/s]
Inserted 2000 rows with 0 errors.


UpdateStatus(num_rows=2000, num_computed_values=26000, num_excs=0, updated_cols=[], cols_with_excs=[])

In [11]:
print(t)

Table 'hani89_asr_data.transcribe_compare'

                     Column Name    Type                               Computed With
                              id  String                                            
                           audio   Audio                                            
                        filename  String                                            
                   transcription  String                                            
                           split  String                                            
                        filePath  String                                            
    whisper_tinyEn_transcription    Json    transcribe(audio=audio, model='tiny.en')
   whisper_smallEn_transcription    Json   transcribe(audio=audio, model='small.en')
  whisper_mediumEn_transcription    Json  transcribe(audio=audio, model='medium.en')
     whisper_large_transcription    Json      transcribe(audio=audio, model='large')
    whisper_baseEn_tr

In [20]:
t.audio.tail(5)

audio
None
None
None
None
None


In [ ]:
from pixeltable.functions.video import extract_audio
from pixeltable.functions.audio import get_metadata
from pixeltable.functions import whisper


In [22]:
## TO DO:  See if this is necessary to create a udf, versus some other mechanism
@pxt.udf
def get_local_audio(filePath: pxt.String, format: pxt.String) -> pxt.Audio:
    return filePath

In [23]:
t.update({
    'audio': get_local_audio(t.filePath, format='wav')}
)

Inserting rows into `transcribe_compare`: 2500 rows [2:10:43,  3.14s/ rows]


UpdateStatus(num_rows=2500, num_computed_values=0, num_excs=0, updated_cols=['transcribe_compare.audio', 'transcribe_compare.whisper_turbo_transcription', 'transcribe_compare.whisper_smallEn_transcription', 'transcribe_compare.whisper_baseEn_transcription', 'transcribe_compare.whisper_tinyEn_transcription', 'transcribe_compare.whisper_large_transcription', 'transcribe_compare.whisper_mediumEn_transcription'], cols_with_excs=[])

In [1]:
#Tiny English
t.add_computed_column( whisper_tinyEn_transcription  = whisper.transcribe(audio=t.audio, model='tiny.en'), if_exists='ignore')  ### HOW COULD I OVERRIDE COLUMN IF I CHANGE MY FUNCTION


#English Small
t.add_computed_column( whisper_smallEn_transcription = whisper.transcribe( audio = t.audio, model = "small.en"), if_exists='ignore')

#English medium
t.add_computed_column( whisper_mediumEn_transcription = whisper.transcribe( audio = t.audio, model = "medium.en"), if_exists='ignore')

#Large
t.add_computed_column( whisper_large_transcription = whisper.transcribe( audio = t.audio, model = "large"), if_exists='ignore')

#Base English model
t.add_computed_column( whisper_baseEn_transcription = whisper.transcribe( audio = t.audio, model = "base.en"), if_exists='ignore')

#Turbo
t.add_computed_column( whisper_turbo_transcription = whisper.transcribe( audio = t.audio, model = "turbo"), if_exists='ignore')


NameError: name 't' is not defined

In [24]:
### add parakeet model
t.tail(5)

id,audio,filename,transcription,split,filePath,whisper_tinyEn_transcription,whisper_smallEn_transcription,whisper_mediumEn_transcription,whisper_large_transcription,whisper_baseEn_transcription,whisper_turbo_transcription
None,,sample_2495.wav,I always feel like I have menstrual pains even if I don't have my period.,train,train_extracted_audio/sample_2495.wav,"{""text"": "" I always feel like half an inch of paint, even if I don't have my period."", ""language"": ""en"", ""segments"": [{""id"": 0, ""end"": 7., ""seek"": 0, ""text"": "" I always feel like half an inch of paint, even if I don't have my period."", ""start"": 0., ""tokens"": [50363, 314, 1464, 1254, 588, 2063, ..., 470, 423, 616, 2278, 13, 50713], ""avg_logprob"": -0.458, ""temperature"": 0., ""no_speech_prob"": 0.034, ""compression_ratio"": 0.986}]}","{""text"": "" I always feel like half-man's remains even if I don't have my period."", ""language"": ""en"", ""segments"": [{""id"": 0, ""end"": 5.36, ""seek"": 0, ""text"": "" I always feel like half-man's remains even if I don't have my period."", ""start"": 0., ""tokens"": [50363, 314, 1464, 1254, 588, 2063, ..., 470, 423, 616, 2278, 13, 50631], ""avg_logprob"": -0.377, ""temperature"": 0., ""no_speech_prob"": 0.027, ""compression_ratio"": 0.972}]}","{""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""language"": ""en"", ""segments"": [{""id"": 0, ""end"": 5.5, ""seek"": 0, ""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""start"": 0., ""tokens"": [50363, 314, 1464, 1254, 588, 314, ..., 470, 423, 616, 2278, 13, 50638], ""avg_logprob"": -0.295, ""temperature"": 0., ""no_speech_prob"": 0.022, ""compression_ratio"": 1.014}]}","{""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""language"": ""en"", ""segments"": [{""id"": 0, ""end"": 5.64, ""seek"": 0, ""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""start"": 0., ""tokens"": [50365, 286, 1009, 841, 411, 286, ..., 380, 362, 452, 2896, 13, 50647], ""avg_logprob"": -0.254, ""temperature"": 0., ""no_speech_prob"": 0.065, ""compression_ratio"": 1.014}]}","{""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""language"": ""en"", ""segments"": [{""id"": 0, ""end"": 5.28, ""seek"": 0, ""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""start"": 0., ""tokens"": [50363, 314, 1464, 1254, 588, 314, ..., 470, 423, 616, 2278, 13, 50627], ""avg_logprob"": -0.291, ""temperature"": 0., ""no_speech_prob"": 0.008, ""compression_ratio"": 1.014}]}","{""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""language"": ""en"", ""segments"": [{""id"": 0, ""end"": 5.36, ""seek"": 0, ""text"": "" I always feel like I have menstrual pains even if I don't have my period."", ""start"": 0., ""tokens"": [50365, 286, 1009, 841, 411, 286, ..., 380, 362, 452, 2896, 13, 50633], ""avg_logprob"": -0.282, ""temperature"": 0., ""no_speech_prob"": 1.934e-11, ""compression_ratio"": 1.014}]}"
None,,sample_2496.wav,I have cut my finger because of playing football and I have to apply pain relief cream but it does not help,train,train_extracted_audio/sample_2496.wav,"{""text"": "" I have cut my finger because of playing football and I have to apply pain relief cream but it does not help."", ""language"": ""en"", ""segments"": [{""id"": 0, ""end"": 5.74, ""seek"": 0, ""text"": "" I have cut my finger because of playing football and I have to apply pain relief"", ""start"": 0., ""tokens"": [50363, 314, 423, 2005, 616, 7660, ..., 423, 284, 4174, 2356, 8259, 50650], ""avg_logprob"": -0.251, ""temperature"": 0., ""no_speech_prob"": 0.028, ""compression_ratio"": 1.174}, {""id"": 1, ""end"": 9.64, ""seek"": 0, ""text"": "" cream but it does not help."", ""start"": 5.74, ""tokens"": [50650, 8566, 475, 340, 857, 407, 1037, 13, 50845], ""avg_lo